In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import re
import string
import os
##from pymorphy3 import MorphAnalyzer
#from stop_words import get_stop_words
import json
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
import numpy as np
from transformers import AutoTokenizer, AutoModel
from sklearn.compose import ColumnTransformer
from sklearn.datasets import fetch_openml
from sklearn.feature_selection import SelectPercentile, chi2
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from transformers import AutoTokenizer, AutoModel
from torch import cuda
from tqdm import tqdm
import numpy as np
import torch.nn.functional as F
import torch
from xgboost import XGBClassifier
np.random.seed(0)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [ ]:
clean_df = pd.read_csv('/content/drive/MyDrive/Znatno_personal/Отчетность ВКР/Артефакты/Parsing_avito/pure_data.csv')

In [ ]:
clean_df = clean_df.drop(columns=['Unnamed: 0'])

In [ ]:
clean_df = clean_df.dropna()

In [ ]:
clean_df = clean_df.drop_duplicates()

In [ ]:
clean_df  = clean_df.reset_index(drop=True)

In [ ]:
symbols_pattern = re.compile(pattern = "["
    "@_!#$%^&*()<>?/\|}{~√•—"
                       "]+", flags = re.UNICODE) #спецсимволы
# двойные пробелы
space_pattern = re.compile('\s+')
#exclude = set(string.punctuation) # удаление пунктуации
def clear_text(text):
    """ Функция удаления спецсимволов"""
    # удаление спецсимволов и emoji
    pre = symbols_pattern.sub(r'',text)

    return space_pattern.sub(' ', pre)

In [ ]:
def preprocess_text(text):
    """ Финальная функция для обработки """
    # srip + lower + punctuation
    sentence = (
        ''.join([x for x in str(text).strip().lower()])
    )

    return clear_text(sentence)

In [ ]:
df1 = pd.read_csv('/content/drive/MyDrive/my_data2 .csv')

In [ ]:
df2 = pd.read_csv('/content/drive/MyDrive/Znatno_personal/Отчетность ВКР/Артефакты/Parsing_avito/my_data22.csv')

In [ ]:
df = pd.concat([df1, df2], ignore_index=True)

In [ ]:
df = df.drop(columns=['Unnamed: 0'])

In [ ]:
values = {'Стаж работы': 0,'Компания': 0, 'Обязанности': 0, "О себе": 0}
df = df.fillna(value=values)

In [ ]:
df = df.drop(index =687)

In [ ]:
df = df.dropna()

In [ ]:
df = df.reset_index(drop=True)

In [ ]:
df

,Идентификатор на сайте,О себе,Стаж работы,Компания,Обязанности,Учебные заведения,Гражданство,Тип занятости,Сфера деятельности,Образование,Пол,Возраст,Оценка
0,№ 3317133582,Опыт работы в косметологии менее года,0.00,Не указано,Не указано,"ММК им С.П. Боткина,Сестринское дело",Россия,"Основная работа, Подработка","Фитнес, салоны красоты",Среднее специальное,Женский,29 лет,1
1,№ 3628798315,"Эстeтичеcкая, инъекциoннaя, аппаратнaя коcмето...",0.00,Не указано,Не указано,"Брянский базовый медицинский колледж,Сестринск...",Россия,Основная работа,"Фитнес, салоны красоты",Среднее специальное,Женский,38 лет,2
2,№ 4525922612,—,0.00,Не указано,Не указано,"Читинский медицинский колледж,Сестринское дело",Россия,Подработка,"Фитнес, салоны красоты",Среднее специальное,Женский,37 лет,1
3,№ 4600063172,"Mеня зoвут Анна, я нa дaнный момент работaю в ...",0.00,Не указано,Не указано,"Ultra pro,Косметолог эстетист",Россия,"Основная работа, Подработка","Фитнес, салоны красоты",Среднее специальное,Женский,39 лет,2
4,№ 4316273702,"Я открытый и честный человек, который относитс...",0.00,Не указано,Не указано,"ПОАНО НИК,Медсестра",Россия,Основная работа,"Фитнес, салоны красоты",Среднее специальное,Женский,24 года,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3074,№ 4445542686,"честность,ответственность,прилежность и органи...",10.08,"сфера красоты, Визажист, Мастер ПМ, бровист , ...",Почти весь спектр услуг,"Калужский медицинский колледж,Лечебное дело",Россия,"Основная работа, Подработка","Фитнес, салоны красоты",Среднее специальное,Женский,30 лет,10
3075,№ 3480822785,Твердые профессиональные знания и опыт.\nОтвет...,5.33,"Красивый бизнес, Косметолог",🔸️Консультирование пациента на первичном приеме,"ООО ""Знание"",Косметолог,Медицинский университе...",Россия,Основная работа,"Фитнес, салоны красоты",Высшее,Женский,51 год,10
3076,№ 3884528092,"Эстетическая косметология ( массажи,пилинги,чи...",10.50,"Самозанятая, Врач-косметолог,ГБУЗ ГП 115, Врач...","Эстетическая и инъекционная косметология,Врач УЗИ","РНИМУ им.Н.И.Пирогова (ординатура ),Дерматовен...",Россия,Основная работа,"Медицина, фармацевтика",Высшее,Женский,31 год,10
3077,№ 2527250246,"Добpoгo врeмeни суток , меня зовут Вeрoника и ...",9.41,"Студия Фабрика 52, Косметолог,Аппаратная косме...","Вся аппаратная косметология, эстетические, инъ...","Национальный открытый институт России,ГМУ,Меди...",Россия,Не указано,"Фитнес, салоны красоты",Высшее,Женский,Не указано,10


In [ ]:
data = data[['message.text', 'Оценка']]

In [ ]:
y.value_counts()

,count
Оценка,
1,849
0,445
7,253
8,239
5,217
2,203
3,156
6,86
4,79


In [ ]:
X = clean_df.drop(columns=['Оценка'])
y = clean_df['Оценка']

In [ ]:
X = X['message.text'].apply(preprocess_text)

In [ ]:
y.value_counts()

,count
Оценка,
1,849
0,445
7,253
8,239
5,217
2,203
3,156
6,86
4,79


In [ ]:
y = y.apply(lambda x: 9 if x == 10 else x)

In [ ]:
y = y.apply(lambda x: x-1 if x > 9 else x)

In [ ]:
y = y - 1

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0,shuffle=True, stratify = y)

In [ ]:
device = 'cuda' if cuda.is_available() else 'cpu'
device

'cuda'

# **Multilingual-e5-large-instruct**

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("intfloat/multilingual-e5-large-instruct")
model = AutoModel.from_pretrained("intfloat/multilingual-e5-large-instruct")
model.to(device)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/690 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

XLMRobertaModel(
  (embeddings): XLMRobertaEmbeddings(
    (word_embeddings): Embedding(250002, 1024, padding_idx=1)
    (position_embeddings): Embedding(514, 1024, padding_idx=1)
    (token_type_embeddings): Embedding(1, 1024)
    (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): XLMRobertaEncoder(
    (layer): ModuleList(
      (0-23): 24 x XLMRobertaLayer(
        (attention): XLMRobertaAttention(
          (self): XLMRobertaSdpaSelfAttention(
            (query): Linear(in_features=1024, out_features=1024, bias=True)
            (key): Linear(in_features=1024, out_features=1024, bias=True)
            (value): Linear(in_features=1024, out_features=1024, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): XLMRobertaSelfOutput(
            (dense): Linear(in_features=1024, out_features=1024, bias=True)
            (LayerNorm): LayerNorm((1024,), eps=1e-05, eleme

In [ ]:
def embed_text(text):
    t = tokenizer(text, padding= True, truncation=True, max_length=512,  return_tensors='pt' )
    with torch.no_grad():
      model_output = model(**{k: v.to(model.device) for k, v in t.items()})
    embeddings = model_output.last_hidden_state[:, 0, :]
    embeddings = torch.nn.functional.normalize(embeddings)
    return embeddings[0].cpu().numpy()

In [ ]:
x = np.zeros((len(X_train), 1024))
y =np.array(y_train)


In [ ]:
for ind, sent in enumerate(tqdm(X_train)):
   x[ind] = embed_text(sent)

100%|██████████| 2021/2021 [00:35<00:00, 57.35it/s]


In [ ]:
x_test = np.zeros((len(X_test), 1024))
y_test =np.array(y_test)

In [ ]:
for ind, sent in enumerate(tqdm(X_test)):
  x_test[ind] = embed_text(sent)

100%|██████████| 506/506 [00:08<00:00, 59.66it/s]


## LogisticRegression


In [ ]:
logistic_reg = LogisticRegression(max_iter=2000)
logistic_reg.fit(x, y)

LogisticRegression(max_iter=2000)

In [ ]:
pred = logistic_reg.predict(x_test)

In [ ]:
accuracy = accuracy_score(y_test, pred)
print("Accuracy for LogisticRegression :", accuracy_score(y_test, pred))
print( "F1-score for LogisticRegression:", f1_score(y_test, pred, average='weighted'))
print("Balanced accuracy forLogisticRegression:", balanced_accuracy_score(y_test,pred))

Accuracy for LogisticRegression : 0.6067193675889329
F1-score for LogisticRegression: 0.540245977487056
Balanced accuracy forLogisticRegression: 0.3847341113691167


## XGB Boosting

In [ ]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.6/383.6 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 7.1 MB/s eta 0:00:00


In [ ]:
import optuna

In [ ]:
def objective(trial):
    params = {
        "objective": "multi:softmax",
        "num_class": 9,
        "eval_metric": "mlogloss",
        "learning_rate": trial.suggest_loguniform("learning_rate", 0.0001, 0.3),
        "max_depth": trial.suggest_int("max_depth", 3, 100),
        "n_estimators": trial.suggest_int("n_estimators", 50, 2000),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.05, 1.0)
    }

    model = XGBClassifier(**params)
    model.fit(x, y)

    y_pred = model.predict(x_test)
    return f1_score(y_test, y_pred, average='weighted')

# Запуск оптимизации
study = optuna.create_study(direction="maximize")  # Ищем max accuracy
study.optimize(objective, n_trials=50)

# Лучшие параметры
print("Лучшие параметры:", study.best_params)

[I 2025-02-27 12:59:45,794] A new study created in memory with name: no-name-90355cc6-31e7-4d6f-b28b-c20f6352f809
<ipython-input-124-e1a597ef903a>:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "learning_rate": trial.suggest_loguniform("learning_rate", 0.0001, 0.3),
[I 2025-02-27 13:09:57,813] Trial 0 finished with value: 0.7048411534752343 and parameters: {'learning_rate': 0.000814776091071527, 'max_depth': 77, 'n_estimators': 1322, 'subsample': 0.870971807823597, 'colsample_bytree': 0.365285844713803}. Best is trial 0 with value: 0.7048411534752343.
<ipython-input-124-e1a597ef903a>:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "learning_rate": trial.suggest_logun

Лучшие параметры: {'learning_rate': 0.013619072808128348, 'max_depth': 80, 'n_estimators': 1475, 'subsample': 0.5532740909498545, 'colsample_bytree': 0.38984815838162784}


In [ ]:
xgb2 = XGBClassifier(objective="multi:softmax", num_class=9, eval_metric="mlogloss", learning_rate = 0.013619072808128348, max_depth = 80, n_estimators = 1475, subsample = 0.5532740909498545, colsample_bytree = 0.38984815838162784)
# Предсказания и оценка
xgb2.fit(x, y)
y_pred = xgb2.predict(x_test)
print("Accuracy:", accuracy_score(y_test, y_pred))

Accuracy: 0.6521739130434783


In [ ]:
print("Accuracy for XGB:", accuracy_score(y_test, y_pred))
print( "F1-score for XGB:", f1_score(y_test, y_pred, average='weighted'))
print("Balanced accuracy for XGB:", balanced_accuracy_score(y_test,y_pred))

Accuracy for XGB: 0.6521739130434783
F1-score for XGB: 0.6326392252925299
Balanced accuracy for XGB: 0.4785479645602923


# **GigarEmbedModel**

In [ ]:
!pip install transformers==4.46.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 2.2 MB/s eta 0:00:00
Reason for being yanked: This version unfortunately does not work with 3.8 but we did not drop the support yet
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 67.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 90.4 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.1
    Uninstalling tokenizers-0.21.1:
      Successfully uninstalled tokenizers-0.21.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.51.2
    Uninstalling transformers-4.51.2:
      Successfully uninstalled transformers-4.51.2


In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 18.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system =

In [ ]:
import os
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModel

In [ ]:
device = 'cuda' if cuda.is_available() else 'cpu'
device

'cuda'

In [ ]:
model1 = AutoModel.from_pretrained('ai-sage/Giga-Embeddings-instruct', trust_remote_code=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/47.8k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.02M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
model1.to(device)

GigarEmbedModel(
  (latent_attention_model): LatentAttentionModel(
    (cross_attend_blocks): ModuleList(
      (0-1): 2 x PreNorm()
    )
  )
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 2048, padding_idx=2)
    (layers): ModuleList(
      (0-26): 27 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=11008, bias=False)
          (up_proj): Linear(in_features=2048, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (in

In [ ]:
from tqdm import tqdm

In [ ]:
def get_embeddings(text):
  task_name_to_instruct = {"example": "получая пример, присвой ему метку класса"}
  query_prefix = task_name_to_instruct["example"] + "\nquestion: "
  queries = [text]
  query_embeddings = model1.encode(queries, instruction=query_prefix)
  query_embeddings = F.normalize(query_embeddings, p=2, dim=1)
  return query_embeddings[0].cpu().numpy()



In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0,shuffle=True, stratify=y)

In [ ]:
x1 = np.zeros((len(X_train), 2048))

In [ ]:
y1 = np.array(y_train)

In [ ]:
for ind, sent in enumerate(tqdm(X_train)):
   x1[ind] =  get_embeddings(sent)

100%|██████████| 2021/2021 [01:34<00:00, 21.32it/s]


In [ ]:
x1_test = np.zeros((len(X_test), 2048))

In [ ]:
y1_test = np.array(y_test)

In [ ]:
for ind, sent in enumerate(tqdm(X_test)):
   x1_test[ind] =  get_embeddings(sent)

100%|██████████| 506/506 [00:23<00:00, 21.20it/s]


## LogisticRegression

In [ ]:
logistic_reg = LogisticRegression(max_iter=2000)
logistic_reg.fit(x1, y1)

LogisticRegression(max_iter=2000)

In [ ]:
pred = logistic_reg.predict(x1_test)

In [ ]:
print("Accuracy for Logistic Regression:", accuracy_score(y1_test, pred))
print("F1 score for Logistic Regression:", f1_score(y1_test, pred, average='weighted'))
print("Balanced accuracy for Logistic Regression:",balanced_accuracy_score(y1_test,pred))

Accuracy for Logistic Regression: 0.5790513833992095
F1 score for Logistic Regression: 0.5214926822990628
Balanced accuracy for Logistic Regression: 0.35960067682571717


## XGB Boosting

In [ ]:
def objective(trial):
    params = {
        "objective": "multi:softmax",
        "num_class": 9,
        "eval_metric": "mlogloss",
        "learning_rate": trial.suggest_loguniform("learning_rate", 0.00001, 0.3),
        "max_depth": trial.suggest_int("max_depth", 3, 100),
        "n_estimators": trial.suggest_int("n_estimators", 50, 1000),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.05, 1.0)
    }

    model = XGBClassifier(**params)
    model.fit(x1, y1)

    y_pred = model.predict(x1_test)
    return f1_score(y1_test, y_pred, average='weighted')

# Запуск оптимизации
study = optuna.create_study(direction="maximize")  # Ищем max f1
study.optimize(objective, n_trials=50)

# Лучшие параметры
print("Лучшие параметры:", study.best_params)

[I 2025-02-28 14:42:04,305] A new study created in memory with name: no-name-03f1da8d-431a-4a3e-8315-fc4ef2301e96
<ipython-input-90-e482197983b8>:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "learning_rate": trial.suggest_loguniform("learning_rate", 0.00001, 0.3),
[I 2025-02-28 14:47:57,058] Trial 0 finished with value: 0.7345585267105077 and parameters: {'learning_rate': 0.010503640552428134, 'max_depth': 45, 'n_estimators': 501, 'subsample': 0.5576026475783387, 'colsample_bytree': 0.40051943982162397}. Best is trial 0 with value: 0.7345585267105077.
<ipython-input-90-e482197983b8>:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "learning_rate": trial.suggest_logu

In [ ]:
xgb = XGBClassifier(objective="multi:softmax", num_class=9, eval_metric="mlogloss", learning_rate = 0.15340579087649192, max_depth = 28, n_estimators = 788, subsample = 0.5355661444792579, colsample_bytree =  0.8338931922625114)
xgb.fit(x1, y1)

# Предсказания и оценка
y_pred = xgb.predict(x1_test)
print("Accuracy for XGB:", accuracy_score(y1_test, y_pred))
print("F1 score for XGB:",f1_score(y_test, y_pred, average='weighted'))
print("Balanced accuracy for XGB:", balanced_accuracy_score(y_test,y_pred))

Accuracy for XGB: 0.6778656126482213
F1 score for XGB: 0.6667144184162868
Balanced accuracy for XGB: 0.541273443274619
